### 13. CNN

- 합성곱 

<img src = 'img/cnn.png'>

- stride : filter를 한번에 얼마나 이동할 것인가? 
- padding : input의 주변을 특정 값으로 감싼다. (ex. zero-padding =1 : 상하좌우에 0을 한줄씩 추가) 

- Conv2d 함수의 구조 
    - torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride = 1, padding = 0, bias = True/False, padding_mode = 0) 
        - in_channels / out_channels : input / output 이미지 갯수
        - kernel_size : kernel(filter) size 

<img src = 'img/conv2d.png'>

- convolution output의 size 

<img src = 'img/output_size.png'>

- pooling은 image size를 줄여준다.
    - max pooling = 2 : 2x2 매트릭스에서 max값만을 가져온다 (4개 --> 1개로 값 변경)
    - average pooling : 2x2 매트릭스에서 평균값만을 가져온다 (4개 --> 1개로 값 변경)
    - 주로 max pooling이 사용되며, 함수는 torch.nn.MaxPool2d(kernel_size) 

- <font color = 'yellow'> CNN for MNIST code

In [1]:
import torch 
import torchvision.datasets as dsets
import torchvision.transforms as transforms 
import random 
import matplotlib.pyplot as plt 
import timeit 
device = 'cpu'

In [11]:
## parameters 
training_epochs = 20 
batch_size = 100 
learning_rate = 0.001 

In [12]:
## dataset and DataLoader
mnist_train = dsets.MNIST(root = 'MNIST_data/', 
                          train = True, 
                          transform = transforms.ToTensor(), 
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/', 
                          train = False, 
                          transform = transforms.ToTensor(), 
                          download = True)

data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                           batch_size = batch_size, 
                                           shuffle = True, 
                                           drop_last = True ) 

<img src = 'img/cnn_layers.png'>

In [13]:
# linear model 대산 CNN Model (2 conv layers)을 만든다 
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        
        ## layer1: 
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        ## layer2: 
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # layer2를 거치면 7x7x64를 얻게되고, 이것이 input이 되어 최종 10개(0~9)로 분류되어야 한다. 
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # size(0)는 모른다, 반복 횟수만큼 나열하라는 뜻
        out = self.fc(out)
        return out

In [14]:
## model, cost, optimizer 
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [15]:
total_batch = len(data_loader)
start_time = timeit.default_timer()  ## start time check 

for epoch in range(training_epochs) : 
    avg_cost = 0 

    for X, Y in data_loader : 
        #X = X.view(-1, 28*28).to(device)  
        X = X.to(device)    #image is already size of (28x28), no reshape need 
        Y = Y.to(device)
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('epoch:', epoch+1, 'cost=', avg_cost)


end_time = timeit.default_timer()  ## end time check 
print('Running time :', end_time - start_time)

epoch: 1 cost= tensor(0.2254, grad_fn=<AddBackward0>)
epoch: 2 cost= tensor(0.0600, grad_fn=<AddBackward0>)
epoch: 3 cost= tensor(0.0453, grad_fn=<AddBackward0>)
epoch: 4 cost= tensor(0.0355, grad_fn=<AddBackward0>)
epoch: 5 cost= tensor(0.0293, grad_fn=<AddBackward0>)
epoch: 6 cost= tensor(0.0244, grad_fn=<AddBackward0>)
epoch: 7 cost= tensor(0.0204, grad_fn=<AddBackward0>)
epoch: 8 cost= tensor(0.0175, grad_fn=<AddBackward0>)
epoch: 9 cost= tensor(0.0143, grad_fn=<AddBackward0>)
epoch: 10 cost= tensor(0.0131, grad_fn=<AddBackward0>)
epoch: 11 cost= tensor(0.0095, grad_fn=<AddBackward0>)
epoch: 12 cost= tensor(0.0088, grad_fn=<AddBackward0>)
epoch: 13 cost= tensor(0.0089, grad_fn=<AddBackward0>)
epoch: 14 cost= tensor(0.0066, grad_fn=<AddBackward0>)
epoch: 15 cost= tensor(0.0061, grad_fn=<AddBackward0>)
epoch: 16 cost= tensor(0.0046, grad_fn=<AddBackward0>)
epoch: 17 cost= tensor(0.0051, grad_fn=<AddBackward0>)
epoch: 18 cost= tensor(0.0047, grad_fn=<AddBackward0>)
epoch: 19 cost= ten

In [30]:
# Test the model using test sets
'''
- test 시에는 gradient가 필요 없으믈 torch.no_grad()로 감싸줌
- batch 없이 1만개의 데이터 한번에 test
- 마찬가지로 to(device)
- prediction size [10000, 10]에 대해 argmax (axis = 1)
- 모든 test 결과에 대해 평균 .mean(), 값만 빼옴 .item()
'''

with torch.no_grad():
    #X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device) ## 784개를 한줄로 뿌려줌 
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item()) 

Accuracy: 0.9860000014305115


In [28]:
print('Running time: {:.4f}'.format(
    (end_time - start_time)/60) + ' min'
     )

Running time: 19.2790 min
